## address
address는 계정의 주소로 20바이트 길이를 가짐.
addresss 객체로서 잔고 조회, 입출금이 가능하며, 이 경우 address payable로 선언해야 함

### 잔고 조회
계정의 잔액 wei 단위를 조회 할 수 있다. 여기서는 누구나 내 잔고를 읽을 수 있다. 
현재 컨트랙의 잔고는 address(this).balance로 구할 수 있다.
this.balance와 같이 하면 this는 현재 인스턴스를 의미하기 때문에 잔고를 구할 수 없다.

Solidity 0.5.0부터는 address(this).balance를 사용해야 한다.
(this는 contract자신의 주소를 말함)
> this.myFunction()

### 송금 
입금은 어느 계좌로든 가능하다. 출금은 오직 자신의 주소에서만 가능하다.
자신의 주소는 unlock, sign할 수 있으니 출금이 가능하다!
금액의 단위는 Wei이다. 
0.5.0버전부터는 주소에서 transfer()함수를 사용하여 송금하려면 address payable로 선언해야 한다.
> address payalbe owner;

- send() 
수신계정으로 송금한다. 송금의 성공or실패 여부는 boolean으로 반환한다.
call stack error 또는 out of gas error 등의 이유로 실패하는 경우에도 예외 Execption이 발생하지 않고 false를 반환하게 된다. 

실패할 경우를 대비하여 require(address.sedn(amount)를 넣어주면 코드가 여기서 중단하게 된다.

receiver.send()함수가 실패되면, fallback 함수가 실행되는데 이때 최대 2300gas를 사용할 수 있다. 2300을 초과하는 금액을 저장 및 송금, 다른 컨트랙을 호출하면 out-of-gas 예외가 발생한다.

throw가 발생하면
- 송신자에게 송금액이 반환되어 원위치로 돌아가게 된다.
- gas는 당연히 소비되었으므로 반환되지 않는다. 
- gas잔액이 남아도 반환되지 않고 모두 소비된 것으로 한다.
- 여러 계정에 송금하는 중 throw 발생 시 나머지 송금계정에는 송금이 아예 발생하지 않는다. 

> address_receiver = 받는 주소;
  if(myAddress.balance>=10) _receiver.send(10);

자신의 잔고가 10(wei)이상이면 받는 주소에게 10만큼 송금하는 것이다.
 

- transfer()
실패하면 throw 예외를 발생시키고, 호출자에게 환급된다. 
따라서 send()를 사용하는 것보다 예외처리에 용이하다.
send()와 마찬가지로 2300gas를 사용할 수 있다.
> address.transfer(amount)

()안에 송금금액을 적는다.


- call()
call, callcode, delegatecall은 가급적 쓰지 않는다. call함수는 gas를 정해서 처리하는 경우에 쓴다. 이 함수는 addr.send(x)와 동일한 기능을 할 수 있다. 단 수신 측에 gas를 사용할 수 있게 하기 때문에 악의적으로 쓰일 수 있다.
> <받는측의 주소>.call.value(20)  

20wei를 송금한다.

> <address>.gas(2).value(20)()
    

# Remix - IntBoolTest 파일

In [2]:
%%writefile src/IntBoolTest.sol
pragma solidity ^0.4.0;

contract IntBoolTest {
    bool married = true;
    uint256 xAge = 22;
    uint256 yAge = 25;
    //fixed phi; // = 3.14;
    function update() public {
        xAge = yAge;
        yAge = 33;
    }
    function setXAge(int age) public {
        xAge = uint(age);  //converstion
    }
    function getXAge() public view returns(uint) {
        return xAge;
    }
    function getYAge() public view returns(uint) {
        return yAge;
    }
    function testInt() public view returns(bool) {
        assert(xAge>=20 && yAge>=20);
        return true;
    }
    function isMarried() public view returns(bool) {
        return married;
    }
}

Writing src/IntBoolTest.sol


# Remix - ByteStringTest 파일

In [3]:
%%writefile src/ByteStringTest.sol

pragma solidity ^0.4.0;
contract BytesStringTest {
    byte b = 0xFF;
    bytes1 b1 = 0xFF;
    bytes2 b2 = 0xFFAA;
    bytes8 place8 = "7 hongji";
    bytes23 place23 = "7 hongji-dong jongro-gu";
    bytes place = "7 hongji-dong jongro-gu Seoul"; //variable length
    bytes myBytes = new bytes(3);  //0x000000
    string constant name = "sha"; //utf-8 string "jsl"
    function getB1() public view returns(byte) {
        return b1;  //byte, so no casting required
    }
    function getB2() public view returns(bytes2) {
        return b2;
    }
    function getB23() public view returns(bytes23) {
        return place23;  //fixed size, value type (no memory)
    }
    /**@return hex bytes. reference type should be set as memory*/
    function getBytes() public view returns(bytes memory) {
        return myBytes;  //smu in hex 0x736d75
    }
    function getLengOfBytes23 () view public returns(uint) {
        return place23.length;  // returns 23
    }
    function getLenOfBytes() pure public returns(uint) {
        bytes memory bm = "7 hongji-dong jongro-gu";
        return bm.length;        // returns 23
    }
    //need the arg in hex e.g. bytes1 0x61 bytes2 0x6161
    //a 61, b 62, ... , y 79
    //try invalid type, e.g. bytes2 0x61 or 0x616161
    function setB2(bytes2 _b2) public {
        b2=_b2;
    }
    function setBytes() public {
        myBytes="smu";
    }
    function getLenOfString() pure public returns(uint) {
        string memory nameLocal="jslLocal";
        //return nameLocal.length;  //error, casting required
        return bytes(nameLocal).length;
    }
    function getString() pure public returns(string memory) {
        string memory s = "한글";
        //bytes memory b4 = "한글";  //ok. bytes is a ref type, so memory used
        return s;
    }
}

Writing src/ByteStringTest.sol


# Remix - AddressTest 파일

In [1]:
%%writefile src/AddressTest.sol
pragma solidity ^0.5.0;
contract AddressTest {
    address owner; // 컨트랙 제작자
    address payable receiver; // 수신측 계정, 돈이 왔다 갔다 할거니까 payable 선언
    uint balanceOfOwner; // 잔고
    constructor() public {
        owner = msg.sender; // 오너 정하기, msg.owner는 전송측 주소
        balanceOfOwner = owner.balance; // 오너는 주소를 말하고, 이 주소의 잔고를 말함
    }
    function deposit() payable public { // 전송자가 보낸 ether를 addressTest에 저장하는 기능 
    }// value에 적은 값이 이 컨트랙으로 이체됨
    function setReceiver(address payable addr) public { // 주소 설정
        receiver = addr;
    }
    function getReceiver() view public returns(address) {
        return receiver; // setReceiver에서 설정한 주소를 반환함
    }
    function getBalanceOfThis() public view returns(uint) { // 컨트랙의 잔고 반환
        return address(this).balance;
    }
    function getBalanceOfOwner() public view returns(uint) { // 송신자의 잔고 반환
        return owner.balance;
    }
    function getBalanceOfReceiver() public view returns(uint) { // 수신자의 잔고 반환
        return receiver.balance;
    }
    function send() public payable { // 수신측으로 send()한다
        require(receiver.send(111));
    }
    function transfer() public payable {
        receiver.transfer(11111); // require을 쓸 수 없음. sender는 return값이 TorF이므로 가능했다.
    }
    function callValue() public payable {
        receiver.call.value(11111)("");
        receiver.call.gas(10).value(11111)("");
    }
}

Writing src/AddressTest.sol


# 2번째 영상

## 저장장소
참조형 타입에는 크게 storage, memory, stack, calldata가 있다.
- storage : 32바이트 길이의 key-value 형식으로 저장되고, 계속 유지되는 경우. 
컨트랙의 모든 함수에서 값을 볼 수 있음. 컨트랙의 상태변수는 기본 값이 storage이며, 일단 저장이 되면 영원히 저장됨.

- memory : 단기간 저장하는 경우 사용됨.
지역변수는 함수 내에서만 짧은 시간 동안 사용되므로, memory 수식어를 붙여줌. 함수 내에서만 짧은 시간 동안 사용되므로 memory 수식어를 붙여줌. 함수의 인자or 반환 값은 Memory를 사용함. 단, struct, array or mapping과 같은 reference type은 지역 변수라 해도 storage가 기본 값임.

- stack : 지역변수(array, stuct, mapping 제외)를 저장하는 stack.
1024수준까지 저장할 수 있고 그 이상은 예외처리 됨. 비용은 memory와 동일함.

- calldata : 외부에서 함수 호출 시 호출자가 제공하는 함수인자를 저장함.

### 요약하면, 상태변수는 storage에 저장. value 타입 지역변수, 함수인자/반환은 memory를 기본 default로 사용함. 단, 함수에서 사용되는 참조타입(struct, array, mapping, string)은 기본이 storage이고, memory를 사용하려면 선언이 필요함

## 가시성
지역변수는 자신이 선언된 함수에 국한되어 쓸 수있다. 반면 멤버변수 state varaviable은 사용할 수 있는 범위를 제어할 수 있다.
- 가시성을 적어주지 않으면 기본 값은 internal로 정의됨. 이는 컨트랙 내부에서만 쓸 수 있고, 상속 시 자식이 물려받을 수 있다.
- external은 외부에서 호출하는 경우에만 허용됨. 내부에서 사용시 this.f()로 호출해야함
- 외부에서 쓰려면 public으로 선언하기. 
- 블록체인 내부에서 컨트랙 자신만 쓸 수 있고, 객체지향 private와 동일한 의미임


In [4]:
%%writefile src/MyBank1.sol
pragma solidity ^0.6.0;

contract MyBank {
    address payable owner;
    uint balance; //contract balance
    constructor() public {
        owner = msg.sender;
        balance = address(this).balance; //contract balance
    }
    
    // value에 적은 값이 이 컨트랙으로 이체됨
    function deposit(uint amount) public payable {
        require(msg.value == amount);// 입금되는 금액은 msg.value라는 전역변수에서 읽어올 수 있음.
        balance += amount;
    }
    
    // 인출하는 함수
    function withdraw(uint amount) public payable {
        balance -= amount; // 인출이니 계좌잔고에서 인출 값(amount)만큼을 빼주어야 함
        owner.transfer(amount); // owner에게 계좌 이체하기
    }
    
    // 계정을 정해서 인출하기 
    function transferTo(address payable receiver, uint amount) public payable { //받는 계정, 송금 금액
        balance -= amount;
        receiver.transfer(amount); //receiver에게 계좌이체 하기
    }
    
    // 잔고를 구하는 함수
    function getBalance() public view returns(uint) {
        return balance;
    }
    
    // 컨트랙의 잔고를 구하는 함수
    function getBalanceOfThis() public  view returns(uint) {
        return address(this).balance;
    }
    
    // 오너의 잔고 구하기
    function getBalanceOfOwner() public view returns(uint) {
        return owner.balance;
    }
}

Writing src/MyBank1.sol


# 3번째 영상

## 형변환
데이터타입의 크기에 대해 주의해야한다. 메모리 절약 방식으로 솔리디티가 사용되기 때문이다.
예를들어 int8에서 int(32bytes)로 변환 시, 작은것에서 큰걸로 변환 시 그냥 괄호안에 써주면 된다.
> int8 i8 = 20;
  uint i256 = int(i8);

uint는 bytes32로 변환가능하다.
> uint x=20;
  bytes32(x);

int는 bytes로 바로 형변환이 불가능하다. int는 int256, 즉 32바이트이므로 bytes32로 변환한 후, 이를 bytes로 변환해야 한다. 바이트마다 옮겨주어야 한다...

ex) 0x68656c6c는 16진수, 8자리이다. 한 자리에는 0,1,2,....,D, E,F까지 2^4 가지를 표현할 수 있고, 4bit가 소요된다. 이 데이터 bytes4를 bytes2로 변환해보자. 오버플로우는 위 4자리부터 잘려 나가게 된다. 따라서 0x6865가 저장된다. 

- string을 bytes로 변환하는 경우
> string s = "hello";
  bytes b = bytes(s);  //0x68656c6c6f
 
address payable은 지급계정으로 사용할 때 payable을 붙이잖아요? 그럼 address로의 형변환은 가능하지만, 반대는 허용되지 않는다. 반대로하려면 중간에 하나를 거쳐야한다. address -> uint160 -> address payable로 20바이트의 정수로 변환한 후 한다.

Q.왜 uint16으로 하지? address가 20byte니까, 16*8하면 160bit가 되기 때문.

In [5]:
%%writefile src/DataConversionTest.sol

pragma solidity ^0.6.0;

contract DataConversionTest {
    function convertInt8ToInt() pure public returns(int) {
        int8 i8 = 20;
        return int(i8); //20
    }
    
    function convertIntToBytes32() pure public returns(bytes32) {
        uint x = 20;
        return bytes32(x); // bytes32로 형변환, 10진수, 0x0000,,,14
    }
    
    function convertInt64ToBytes8() pure public returns(bytes8) {
        uint64 x = 10;
        return bytes8(x); // 0x0000...aa
    }
    
    function convertIntToBytes() pure public returns(bytes memory) {
        uint x = 20;
        bytes32 y = bytes32(x);
        bytes memory z = new bytes(32); // 데이터를 받을 bytes, bytes는 참조형이므로 memory가 되어야 함
        for(uint i=0; i<32; i++) {
            z[i] = y[i];
        }
        return z; // 0 is 60.
    }
    
    function convertStringToBytes() pure public returns(bytes memory) {
        string memory s = "hello";
        return bytes(s);
    }
    
    function convertBytes4ToBytes2() pure public returns(bytes2) {
        bytes4 b4 = 0x68656c6c;
        return bytes2(b4); //overflow occur
    }
    
    function convertBytes4ToInt32() pure public returns(int32) {
        bytes4 b4 = 0x68656c6c;
        return int32(b4); //16 -> 10, no hex, int 10
    }
}

Writing src/DataConversionTest.sol


## 전역변수
- 화폐단위 wei 10^18, 1ether는 1000000000000000000이다.
> 1ether == 1000 finney
- 시간 years는 지원되지 않으며, seconds로 표시됨. now는 현재시간, block.timestamp와 동일함
> 1 == 1 seconds, 1 days == 86400 seconds

- 트랜잭션 관련 전역변수 (tx)
  - tx.origin 트랜잭션에 사인한 계정, msg.sender와 같은 의미임.
  - tx.gasprice 트랜잭션 호출자가 명시한 gas price
  
- msg 컨트랙의 함수를 호출한 전송 관련 정보
  - msg.data : call데이터 (bytes값으로 표현)
  - msg.gas : gas 잔여분
  - msg.sender : 현재 함수를 호출하는 측의 주소
     - 거래가 호출자 O1 -> 컨트랙 C1 -> 컨트랙C2의 순서대로 완성될 경우 :
     - C2에서는 msg.sender는 바로 직전의 호출자 C1을 말하며, 따라서 컨트랙 가능함
     - 반면에 tx.origin은 컨트랙이 될 수 없음
  - msg.value : 컨트랙에 지급되는 ether(단위는 wei)

- block
  - block.coinbase : 현재 블록의 마이너의 주소, 즉 지급 계정
  - block.difficulty : 현재 블록의 난이도
  - block.gaslimit : 현재 블록의 gaslimit
  - block.number : 현재 블록의 수
  - block.blockhash : 현재 블록 해쉬 값, 블록에서 블록에 붙여진 해쉬값
  - block.timestamp : 현재 블록 타임스탬프 epoch(경과된 수)(1970년 1월 1일 0시) 이후 지나간 초, uint256
  


In [7]:
%%writefile src/GlobalVarsTest.sol

pragma solidity ^0.6.0;
contract GlobalVarsTest {
    function getEther() pure public returns(uint) {
        return 1 ether;  // 전역변수, 1000000000000000000
    }
    
    function getDays() view public returns(uint,uint) {
        require(block.timestamp==now); // 예외문, 같으면 다음문장을 수행함
        return(now, 1 days);
    }
    
    function getMsgValue() view internal returns(uint) {
        return msg.value; // 금액이 들어간 필드를 의미함. 
    }
    
    function getMsgSender() view public returns(address) {
        return msg.sender;
    }
    
    function getCoinbase() view public returns(address) {
        return block.coinbase;
    }
    
    function getBlockNumber() view public returns(uint) {
        return block.number;
    }
    
    function getBlockTimeStamp() view public returns(uint) {
        return block.timestamp;
    }
}

Writing src/GlobalVarsTest.sol
